In [13]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('Spark_sql') \
    .getOrCreate()

23/06/01 13:42:28 WARN Utils: Your hostname, Nguyen-Duc-Trung resolves to a loopback address: 127.0.1.1; using 192.168.130.52 instead (on interface eth0)
23/06/01 13:42:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/01 13:42:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('../data/green/*.parquet')

In [4]:
df_green.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [5]:
df_green.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       1| 2022-03-01 07:24:14|  2022-03-01 07:34:04|                 N|       1.0|          74|         151|            1.0|          2.3|        9.5|  0.5|    0.

In [6]:
df_yellow = spark.read.parquet('../data/yellow/*.parquet')
df_yellow.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [7]:
#Change the column names : pickup_datetime, dropoff_datetime for the same columns in both dataframes 
df_green = df_green \
        .withColumnRenamed("lpep_pickup_datetime", "pickup_datetime") \
        .withColumnRenamed("lpep_dropoff_datetime", "dropoff_datetime")

df_yellow = df_yellow \
        .withColumnRenamed("tpep_pickup_datetime", "pickup_datetime") \
        .withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")


In [12]:
len(df_green.columns), len(df_yellow.columns)

(20, 19)

In [15]:
#find which columns are different
set(df_green.columns) - set(df_yellow.columns)

{'ehail_fee', 'trip_type'}

In [16]:
df_green.select('ehail_fee','trip_type').distinct().show()

+---------+---------+
|ehail_fee|trip_type|
+---------+---------+
|     null|     null|
|     null|      1.0|
|     null|      2.0|
+---------+---------+



In [17]:
set(df_yellow.columns) - set(df_green.columns)

{'airport_fee'}

In [18]:
df_yellow.select('airport_fee').distinct().show()

+-----------+
|airport_fee|
+-----------+
|        0.0|
|       1.25|
|       null|
|      -1.25|
|        0.5|
+-----------+



In [19]:
#Check the common columns between the two dataframes    
set(df_green.columns).intersection(set(df_yellow.columns))

#create a list of common columns
common_columns = list(set(df_green.columns).intersection(set(df_yellow.columns)))

In [20]:
#Add a new column to each dataframe to identify the company (yellow or green) : name column as service type
df_green_sel = df_green \
        .select(common_columns) \
        .withColumn('service_type', F.lit('green'))

df_yellow_sel = df_yellow \
        .select(common_columns) \
        .withColumn('service_type', F.lit('yellow'))

In [24]:
#Combine the two dataframes into one
df_trips = df_green_sel.union(df_yellow_sel)

In [25]:
#Combine by uniall and compare two new dataframes
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [26]:
df_trips.count(), df_trips_data.count()

(36821089, 36821089)

In [27]:
#Count the numbers of rows for each service type
df_trips.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green|  840402|
|      yellow|35980687|
+------------+--------+



In [28]:
#Create temporary view for the dataframe df_trips in order to use SQL queries
df_trips.createOrReplaceTempView("trips_data")

In [29]:
spark.sql("SELECT service_type, count(*) as count FROM trips_data GROUP BY service_type").show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green|  840402|
|      yellow|35980687|
+------------+--------+



In [30]:
spark.sql("Describe trips_data").show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|        PULocationID|   bigint|   null|
|          tip_amount|   double|   null|
|     pickup_datetime|timestamp|   null|
|        tolls_amount|   double|   null|
|  store_and_fwd_flag|   string|   null|
|    dropoff_datetime|timestamp|   null|
|improvement_surch...|   double|   null|
|congestion_surcharge|   double|   null|
|             mta_tax|   double|   null|
|               extra|   double|   null|
|       trip_distance|   double|   null|
|        DOLocationID|   bigint|   null|
|        payment_type|   double|   null|
|     passenger_count|   double|   null|
|         fare_amount|   double|   null|
|        total_amount|   double|   null|
|          RatecodeID|   double|   null|
|            VendorID|   bigint|   null|
|        service_type|   string|   null|
+--------------------+---------+-------+



In [31]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone, -- Rename PULocationID column to revenue_zone
    date_trunc('month', pickup_datetime) AS revenue_month, -- Truncate pickup_datetime to month level
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare, -- Calculate monthly fare revenue
    SUM(extra) AS revenue_monthly_extra, -- Calculate monthly extra revenue
    SUM(mta_tax) AS revenue_monthly_mta_tax, -- Calculate monthly MTA tax revenue
    SUM(tip_amount) AS revenue_monthly_tip_amount, -- Calculate monthly tip revenue
    SUM(tolls_amount) AS revenue_monthly_tolls_amount, -- Calculate monthly tolls revenue
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge, -- Calculate monthly improvement surcharge revenue
    SUM(total_amount) AS revenue_monthly_total_amount, -- Calculate monthly total revenue
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge, -- Calculate monthly congestion surcharge revenue

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count, -- Calculate average monthly passenger count
    AVG(trip_distance) AS avg_montly_trip_distance -- Calculate average monthly trip distance
FROM
    trips_data -- Select data from trips_data
GROUP BY
    1, 2, 3 -- Group results by revenue_zone, revenue_month, and service_type
""")

In [33]:
df_result.show()

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|revenue_monthly_congestion_surcharge|avg_montly_passenger_count|avg_montly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|          92|2022-

In [34]:
type(df_result)

pyspark.sql.dataframe.DataFrame

In [35]:
df_result.write.parquet('../data/report/revenue-2022/')

In [36]:
data_trips = spark.read.parquet('../data/report/revenue-2022/')

In [37]:
data_trips.printSchema()

root
 |-- revenue_zone: long (nullable = true)
 |-- revenue_month: timestamp (nullable = true)
 |-- service_type: string (nullable = true)
 |-- revenue_monthly_fare: double (nullable = true)
 |-- revenue_monthly_extra: double (nullable = true)
 |-- revenue_monthly_mta_tax: double (nullable = true)
 |-- revenue_monthly_tip_amount: double (nullable = true)
 |-- revenue_monthly_tolls_amount: double (nullable = true)
 |-- revenue_monthly_improvement_surcharge: double (nullable = true)
 |-- revenue_monthly_total_amount: double (nullable = true)
 |-- revenue_monthly_congestion_surcharge: double (nullable = true)
 |-- avg_montly_passenger_count: double (nullable = true)
 |-- avg_montly_trip_distance: double (nullable = true)



In [38]:
#Check the revenue with the service type green and yellow
data_trips.groupBy('service_type').sum('revenue_monthly_total_amount').show()

+------------+---------------------------------+
|service_type|sum(revenue_monthly_total_amount)|
+------------+---------------------------------+
|       green|             1.6238446400000075E7|
|      yellow|              7.776290886601588E8|
+------------+---------------------------------+



In [41]:
#Check the revenue with the service type green and yellow with sql query
data_trips.createOrReplaceTempView("trips_data")

spark.sql("SELECT service_type, round(sum(revenue_monthly_total_amount),2) as revenue FROM trips_data GROUP BY service_type").show()

+------------+--------------+
|service_type|       revenue|
+------------+--------------+
|       green|  1.62384464E7|
|      yellow|7.7762908866E8|
+------------+--------------+



In [46]:
#Check the revenue of each month with the service type green in sql query
spark.sql("""
SELECT
    service_type,
    date_trunc('month', revenue_month) AS revenue_month,
    round(sum(revenue_monthly_total_amount),2) as revenue
FROM    
    trips_data
WHERE 
    service_type = 'green' and revenue_month > '2019-12-31'
GROUP BY
    1, 2
ORDER BY
    2""").show()


+------------+-------------------+----------+
|service_type|      revenue_month|   revenue|
+------------+-------------------+----------+
|       green|2022-01-01 00:00:00|1080941.04|
|       green|2022-02-01 00:00:00| 1234238.7|
|       green|2022-03-01 00:00:00|1431190.96|
|       green|2022-04-01 00:00:00|1428324.72|
|       green|2022-05-01 00:00:00|1511832.43|
|       green|2022-06-01 00:00:00| 1460743.9|
|       green|2022-07-01 00:00:00|1264161.54|
|       green|2022-08-01 00:00:00| 1308519.8|
|       green|2022-09-01 00:00:00| 1437858.2|
|       green|2022-10-01 00:00:00|1362377.95|
|       green|2022-11-01 00:00:00|1179966.96|
|       green|2022-12-01 00:00:00|1516182.29|
|       green|2023-01-01 00:00:00|  21994.45|
+------------+-------------------+----------+

